In [1]:
import pandas as pd
import numpy as np

# Standardized Options

[Locations](#Locations): `loc_list`, `loc_prob_dict`, `loc_abbr_dict`

[Languages](#Languages): `lg_list`, `lg_prob_dict`

[Services](#Services): `sv_adv_list`, `sv_con_list`

[Names](#Names): `first_nm_list`, `last_nm_list`

## Locations (States) <a id='Locations'></a>

In [2]:
df_st = pd.read_csv('data/bank_branch_master_list.csv')
# df.head()

In [3]:
# # find bank name
# names = np.sort(df_st.NAME.unique())
# for name in names:
#     print(name)
# # Bank Of America, National Association

In [4]:
# # find column index
# for i, col in enumerate(df_st.columns):
#     print(i, col)
# # 20 - NAME
# # 24 - SERVTYPE
# # 25 - STALP
# # 27 - STNAME

In [5]:
# select BoA branches with specific service types
df_st_sub = df_st.loc[(df_st['NAME']=='Bank Of America, National Association') 
                & (df_st['SERVTYPE'].isin([11,12,15,16])) 
                & (df_st['STNAME']!=' ')].iloc[:,np.r_[20,24,25,27]].copy()
df_st_sub.reset_index(drop=True, inplace=True)
# SERVTYPE
#     11 Full Service Brick and Mortar Office
#     12 Full Service Retail Office
#     15 Full Service Home/Phone Banking
#     16 Full Service Seasonal Office
# df_st_sub.head()

In [6]:
# create location objects
states = df_st_sub.STALP.unique()
total = len(df_st_sub)

loc_list = list(states) # state list
loc_prob_dict = {} # state:proportion
loc_abbr_dict = {} # state:statename

for state in states:
    count = len(df_st_sub.loc[df_st_sub['STALP']==state])
    loc_prob_dict[state] = count/total
    loc_abbr_dict[state] = df_st_sub.loc[df_st_sub['STALP']==state].iloc[0,3]

# loc_prob_dict
# loc_abbr_dict
# loc_list

## Languages <a id='Languages'></a>

In [7]:
df_lg = pd.read_excel('data/languages.xlsx','Probability')
df_lg.head()

,Language,Number of Speakers,Probability
0,English,291484482,1.000000
1,Spanish,37458624,0.703823
2,Chinese,2896766,0.054428
3,Tagalog,1613346,0.030314
4,Vietnamese,1399936,0.026304


In [8]:
# create language objects
lg_list = list(df_lg.iloc[1:,0])

lg_prob_dict = {} # language:proportion
for language in lg_list:
    lg_prob_dict[language] = df_lg.loc[df_lg['Language']==language].iloc[0,2]

# lg_list
# lg_prob_dict

## Services <a id='Services'></a>

In [9]:
df_sv = pd.read_excel('data/services.xlsx','categories')
# df_sv.head()

In [10]:
# create service objects
sv_adv_list = list(df_sv.columns) # list of services provided by advisors, category level
sv_con_list = [] # list of services available for consumers, service level

for i in range(len(sv_adv_list)):
    col_arr = df_sv.iloc[:,i].dropna()
    for j in range(len(col_arr)):
        sv_con_list.append(sv_adv_list[i]+'-'+col_arr[j])

# sv_adv_list
# sv_con_list

## Names <a id='Names'></a>

In [11]:
df_nm = pd.read_excel('data/names.xlsx','names')
# df_nm.head()

In [12]:
# create name objects
first_nm_list = list(df_nm.iloc[:,0])
last_nm_list = list(df_nm.iloc[:,1])

# first_nm_list
# last_nm_list

# Mock Data Generation
## Financial Advisors

In [13]:
def random_select(array_input, sample_size):    
    optn_list = []
    prob_list = []
    
    for i in array_input:
        optn_list.append(i)
        
        if type(array_input) == dict:   
            prob_list.append(array_input[i])

        else:
            prob_list.append(1/len(array_input))
        
    return np.random.choice(a = optn_list, size = sample_size, replace = False, p = prob_list)

In [27]:
input_dict = {'first_name':(first_nm_list,1) # category:object,samplesize
              ,'last_name':(last_nm_list,1)
              ,'language':(lg_prob_dict,2)
              ,'service':(sv_adv_list,3)
              ,'location':(loc_prob_dict,1)}

adv_record = [] # name, English, language1, language2, service1, service2, service3, location

for category in input_dict:
    for selection in random_select(input_dict[category][0],input_dict[category][1]):        
        if category == 'last_name':
            adv_record[0] = adv_record[0]+' '+selection
            adv_record.append('English') # add English as default           
        else:
            adv_record.append(selection)
    print(adv_record)

['Jameela']
['Jameela Nason', 'English']
['Jameela Nason', 'English', 'Spanish', 'Korean']
['Jameela Nason', 'English', 'Spanish', 'Korean', 'Home Loan', 'Investment', 'Business']
['Jameela Nason', 'English', 'Spanish', 'Korean', 'Home Loan', 'Investment', 'Business', 'FL']
